# ML Pipeline Preparation
Create the ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [34]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [31]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
#df = pd.read_sql('SELECT * FROM InsertDatabaseName', engine)
df = pd.read_csv('InsertDatabaseName.csv', dtype = {'message': str, 'original': str, 'genre': str})
X = df["message"]
y = df.iloc[:, 4:]

In [24]:
X.head()

0    b'Weather update - a cold front from Cuba that...
1           b'Is the Hurricane over or is it not over'
2                   b'Looking for someone but no name'
3    b'UN reports Leogane 80-90 destroyed. Only Hos...
4    b'says: west side of Haiti, rest of the countr...
Name: message, dtype: object

In [32]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    return word_tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [29]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(random_state = 42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [52]:
y_pred = pd.DataFrame(data = pipeline.predict(X_test), columns = y.columns)

In [113]:
precision, recall, f1_score = [], [], []

for category in y.columns:
    scores = classification_report(y_test[category], y_pred[category])
    precision.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") if len(x) > 0][:3][0])
    recall.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") if len(x) > 0][:3][1])
    f1_score.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") if len(x) > 0][:3][2])    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [123]:
classirication_metric = pd.DataFrame(data = [precision, recall, f1_score], 
                                     index = ["precision", "recall", "f1_score"], columns = y.columns)#.T

classirication_metric

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
precision,0.77,0.87,0.99,0.74,0.91,0.94,0.97,0.98,0.95,1.00,...,0.97,0.93,0.84,0.94,0.92,0.99,0.96,0.98,0.92,0.85
recall,0.79,0.88,1.00,0.74,0.93,0.95,0.97,0.98,0.96,1.00,...,0.99,0.96,0.84,0.94,0.93,0.99,0.96,0.98,0.94,0.86
f1_score,0.77,0.86,0.99,0.73,0.90,0.93,0.96,0.98,0.95,1.00,...,0.98,0.94,0.83,0.93,0.92,0.98,0.96,0.97,0.92,0.83


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.